In [59]:
# Automatic reload of local libraries
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
!python -u train.py --params=params_local.json

Namespace(batch_size=3, dmax_len=10000, docs='stub_data/docs.csv', drop_prob=0.6, embeddings='/home/mrim/data/embeddings/cc.ru.300.bin', epochs=2, inverted_index='./inverted_index.csv', is_stub=1, layers=[300, 200, 5000], learning_rate=5e-05, output_file='./model.pth', params='params_local.json', qmax_len=100, qrels='stub_data/qrels', queries='stub_data/queries.csv', reg_lambda=1e-07, summary_folder='./snrm-summary/')
Running....
Device to use: cpu
Preprocessing data started...
   index   0  1   2  3
0     10  10  0  11  1
1      2   2  0   3  1
2      8   8  0   9  1
3      1   1  0   2  1
4      7   7  0   8  1
5      9   9  0  10  1
6      3   3  0   4  1
7      0   0  0   1  1
8      5   5  0   6  1
Finished.
Training, epoch # 0
(3, 100, 300)


Traceback (most recent call last):
  File "train.py", line 113, in <module>
    run(args)
  File "train.py", line 96, in run
    train_and_validate(args, model, mi_generator)
  File "train.py", line 55, in train_and_validate
    train_loss = model.train(train_batch)
  File "C:\Users\sonya\Desktop\snrm-bert\snrm\snrm.py", line 123, in train
    loss.backward()
  File "C:\Users\sonya\Anaconda3\lib\site-packages\torch\tensor.py", line 118, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "C:\Users\sonya\Anaconda3\lib\site-packages\torch\autograd\__init__.py", line 87, in backward
    grad_tensors = _make_grads(tensors, grad_tensors)
  File "C:\Users\sonya\Anaconda3\lib\site-packages\torch\autograd\__init__.py", line 28, in _make_grads
    raise RuntimeError("grad can be implicitly created only for scalar outputs")
RuntimeError: grad can be implicitly created only for scalar outputs


In [61]:
from utils.msmarco.train_loader import TrainLoader

In [65]:
train_loader = TrainLoader(docs = "/home/mrim/data/collection/msmarco/documents/msmarco-docs.tsv.gz", \
                           train_queries = "/home/mrim/data/collection/msmarco/documents/train/msmarco-doctrain-queries.tsv", \
                           train_qrels = "/home/mrim/data/collection/msmarco/documents/train/msmarco-doctrain-qrels.tsv", \
                           valid_queries =  "/home/mrim/data/collection/msmarco/documents/dev/msmarco-docdev-queries.tsv", \
                           valid_qrels = "/home/mrim/data/collection/msmarco/documents/dev/msmarco-docdev-qrels.tsv", \
                           save_mem = True)

ParserError: Error tokenizing data. C error: Expected 44 fields in line 3, saw 118


In [ ]:
df_docs = pd.read_csv("/home/mrim/data/collection/msmarco/documents/msmarco-docs-cut.tsv", sep='\t', header=None, names = ["id_right", "text_right"], na_filter=False)

In [85]:
df_docs = df_docs.loc[:, ['id_right', 'text_right']]

In [86]:
df_docs.to_csv("/home/mrim/data/collection/msmarco/documents/msmarco-docs-cut.tsv", sep='\t', index=False, header=False)

In [88]:
df_docs.head()

,id_right,text_right
0,D1555982,Science & Mathematics Physics The hot glowing ...
1,D301595,School-Age Kids Growth & Development Developme...
2,D1359209,Check for Lice Nits Edited by Mian Sheilette O...
3,D2147834,Global Business Dubai Opens a Tower to Beat Al...
4,D1568809,"Coulterville, CA Real Estate & Homes for Sale4..."


In [9]:
import pandas as pd

df_valid_qrels = pd.read_csv("/home/mrim/data/collection/msmarco/documents/dev/msmarco-docdev-qrels.tsv", sep=" ", na_filter=False, header=None)

In [10]:
df_valid_qrels.head()

,0,1,2,3
0,2,0,D1650436,1
1,1215,0,D1202771,1
2,1288,0,D1547717,1
3,1576,0,D1313702,1
4,2235,0,D2113408,1


In [19]:
df_valid_queries = pd.read_csv("/home/mrim/data/collection/msmarco/documents/test/msmarco-test2019-queries.tsv", sep='\t', header=None, names = ["id_left", "text_left"], na_filter=False)
df_valid_queries.head()

,id_left,text_left
0,1108939,what slows down the flow of blood
1,1112389,"what is the county for grand rapids, mn"
2,792752,what is ruclip
3,1119729,what do you do when you have a nosebleed from ...
4,1105095,where is sugar lake lodge located


In [18]:
from utils.pytrec_evaluator import read_qrels

In [15]:
qrels = read_qrels("/home/mrim/data/collection/msmarco/documents/test/2019qrels-docs.txt")

In [16]:
qrels

defaultdict(dict,
            {'19335': {'D1035833': 0,
              'D1068065': 0,
              'D1068066': 0,
              'D1088': 0,
              'D109632': 2,
              'D109633': 1,
              'D1106697': 0,
              'D1110666': 0,
              'D1139898': 0,
              'D114440': 0,
              'D1148280': 0,
              'D1162924': 0,
              'D1191436': 0,
              'D1191437': 0,
              'D119997': 0,
              'D119998': 0,
              'D121354': 0,
              'D1234328': 0,
              'D1246864': 0,
              'D1246866': 0,
              'D1252732': 0,
              'D1254937': 1,
              'D1261628': 0,
              'D1269459': 0,
              'D1273654': 0,
              'D1291795': 0,
              'D1291797': 0,
              'D1325600': 0,
              'D1340976': 0,
              'D1347090': 1,
              'D1348226': 0,
              'D1348227': 0,
              'D1348228': 0,
              'D1360501':